# Day 11
Find the description of the problem [here](https://adventofcode.com/2025/day/11)!

## Part 1

Puzzle input:

In [354]:
with open("input_files/day_11.txt") as input_file:
    input = input_file.read()

Test input:

In [355]:
# # Comment this cell to use the puzzle input instead of the test input
# input = """aaa: you hhh
# you: bbb ccc
# bbb: ddd eee
# ccc: ddd eee fff
# ddd: ggg
# eee: out
# fff: out
# ggg: out
# hhh: ccc fff iii
# iii: out"""

Parse the input:

In [356]:
device_list = {}
for line in input.splitlines():
    device, output = line.split(": ")
    device_list[device] = output.split(" ")

Define a recursive search function that returns the total amount of paths:

In [357]:
def search(device, device_list):
    outputs = device_list[device]
    if "out" in outputs:
        return 1
    outputs_value = 0
    for output in outputs:
        outputs_value += search(output, device_list)
    return outputs_value

In [358]:
different_paths = search("you", device_list)
print(f"There are {different_paths} different paths leading from 'you' to 'out'.")

There are 643 different paths leading from 'you' to 'out'.


## Part 2

New test input:

In [359]:
# # Comment this cell to use the puzzle input instead of the test input
# input = """svr: aaa bbb
# aaa: fft
# fft: ccc
# bbb: tty
# tty: ccc
# ccc: ddd eee
# ddd: hub
# hub: fff
# eee: dac
# dac: fff
# fff: ggg hhh
# ggg: out
# hhh: out"""

Parse the new input:

In [360]:
device_list = {}
for line in input.splitlines():
    device, output = line.split(": ")
    device_list[device] = output.split(" ")

I changed the search function to have a variable end and to avoid one device (more on this later). After this, the solution was taking too long to compute (see the end result), so I thought that a cache would solve the issue to avoid recomputing values, and this worked.

In [361]:
def search(device, device_list, end, avoid, cache=None):
    if cache is None:
        cache = {}
    
    if device in cache:
        return cache[device]
    
    outputs = device_list[device]
    outputs_value = 0
    for output in outputs:
        if output == "out":
            if end == "out":
                outputs_value += 1
        elif output == end:
            outputs_value += 1
        elif output == avoid:
            pass
        else:
            outputs_value += search(output, device_list, end, avoid, cache)
    cache[device] = outputs_value
    return outputs_value

I didn't know which came first, `fft` or `dac`, so I computed both possibilities and took the one that was not 0.

The approach I followed was to search all possible paths to the first node, then all the paths from the first node to the second node, and finally all the paths to the `out` node. To avoid overcounting and separating the paths that went directly from `svr` to the first node without going into the second node, I implemented the `avoid` feature in the `search` function. I'm not entirely sure if it's necessary, as the first node is fully before the second node, but if it works, it works!

In [362]:
different_paths_1 = search("svr", device_list, "fft", "dac") * search("fft", device_list, "dac", "out") * search("dac", device_list, "out", "dac")
different_paths_2 = search("svr", device_list, "dac", "fft") * search("dac", device_list, "fft", "out") * search("fft", device_list, "out", "dac")
print(f"There are {max(different_paths_1, different_paths_2)} different paths leading from 'svr' to 'out' while visiting 'dac' and 'fft'.")

There are 417190406827152 different paths leading from 'svr' to 'out' while visiting 'dac' and 'fft'.
